In [4]:
import os 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten

In [ ]:
#Find train and test data.
train_path = os.path.abspath('train.csv')
test_path = os.path.abspath('test.csv')

In [5]:
#Read the train and test data.
train=pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [6]:
X=train.drop('label',axis=1).values.astype('float32')
y=train['label'].values.astype('int32')

In [7]:
#Change y data type to categorical
from tensorflow.keras import utils
y = utils.to_categorical(y) 

In [10]:
#Add rotations, zoom, shift, rescale to the training/testing data.
datagen_train = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        rescale=1/255.)

datagen_test = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        rescale=1/255.)

In [11]:
X_train,X_test,y_train, y_test=train_test_split(X,y,random_state=0)


#expand 1 more dimention as 1 for colour channel gray
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)

In [12]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(28,28,1)),

    layers.Conv2D(filters=32,kernel_size=3,activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=32,kernel_size=3,activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=32,kernel_size=5,strides=2,padding='same',activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(64, kernel_size = 3, activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size = 3, activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(128, kernel_size = 3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(10, activation='softmax'),
])
    

In [13]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],)

datagen_train.fit(X_train)
model.fit_generator(datagen_train.flow(X_train, y_train, batch_size = 32),
                    validation_data = (X_test, y_test), steps_per_epoch = len(X_train) // 32,
                    epochs = 10)

# early_stopping = keras.callbacks.EarlyStopping(
#     patience=5,
#     min_delta=0.01,
#     restore_best_weights=True,
# )

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
#    callbacks=[early_stopping],
    batch_size=64,
    epochs=150,
)

Next, we look at how our loss and accuracy of the training data compare to the loss and accuracy of our test data.

In [14]:
history_df = pd.DataFrame(history.history)
# history_df[['loss', 'val_loss']].plot(title="Loss: Training data vs validation data")
# history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy: Training vs validation data")

Here is the validation accuracy, validation loss, training accuracy, and training loss

In [15]:
from IPython.display import Image
Image("../input/history/Dashboard 1.png")

In [16]:
test= test.values.astype('float32')
test = test.reshape(test.shape[0], 28, 28,1)
label=model.predict_classes(test)

In [17]:
my_submission = pd.DataFrame({'ImageId': list(range(1,len(label)+1)), 'Label': label})
my_submission.to_csv('submission.csv', index=False)